In [1]:
import xarray
import numpy as np
import sys
sys.path.insert(0,'../code/')
from lme_forecast_general import LME
import pandas as pd

/Users/jizezhang/miniconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/jizezhang/miniconda3/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


#### Read data

In [2]:
data = pd.read_csv('./data/mp_ratio.csv')
data = data.sort_values(by=['location_id','age_group_id','sex_id','year_id'])
data = data.iloc[:,1:]

In [3]:
data.head()

,age_group_id,location_id,sex_id,year_id,scenario,value,haq,intercept,lme_fit
0,13,6,1,1990,0,-2.891245,43.258941,1,-3.332449
1,13,6,1,1991,0,-2.897362,44.078531,1,-3.334340
2,13,6,1,1992,0,-2.875354,44.913657,1,-3.336268
3,13,6,1,1993,0,-2.870614,45.764612,1,-3.338232
4,13,6,1,1994,0,-2.881733,46.631696,1,-3.340233


In [4]:
Y = data['value'].values
haq = data[(data['age_group_id'] == 13) & (data['sex_id'] == 1)]['haq'].values
haq = haq - np.mean(haq)
n_locs = 195
n_ages = len(set(data['age_group_id'].values))
T = 28

#### Build model

$$ y = \beta \text{HAQ} + \beta_I \text{age-sex} + \pi_{0,l,a} + \pi_{1,l,a}\text{HAQ} $$

In [6]:
model = LME([n_locs,n_ages,2,T], 2, Y, [(haq,[n_locs,1,1,T])], indicator=[1,n_ages,2,1], global_ids=[0], 
            ran_list=[(0,[1,1])],global_intercept=False, ran_intercepts=[[1,1]])

In [7]:
import time
t0 = time.time()
model.optimize(inner_max_iter=500)
print('elapsed', time.time()-t0)

n_groups 2340
k_beta 25
k_gamma 2
total number of fixed effects variables 28
fit with gamma fixed...
finished...elapsed 3.7199220657348633
elapsed 97.9530680179596


In [8]:
lme_fit = data['lme_fit'].values
np.linalg.norm(model.yfit_no_random - lme_fit) 

9.38198055352432